# Population density of Melbourne city by statistical area

In [1]:
!pip install osmnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 2.5 MB/s eta 0:00:00


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#import the packages required for this task

import geopandas as gpd
import folium
import numpy as np
import osmnx as ox
import matplotlib.pyplot as plt

In [6]:
#read in the shape file a gdf

shp_file_path_mel = "/content/drive/MyDrive/FinalProject/Data/Pop_Density_Data/Mel/SA1_AUS_Data.shp"
gdf_mel = gpd.read_file(shp_file_path_mel)

In [7]:
#change the crs to be 3112 to algin with the projection used during the other anaylsis for Melbourne

gdf_mel = gdf_mel.to_crs(epsg=3112)

In [8]:
#create a attribute and calculate the population density in it.
#population density can be calculated by number of people / area
#for each SA1 we have calulcated the pop density
#the units for Pop_density will be people/km^2. Given the suits for AREA_ALBER are in km^2

gdf_mel['Pop_density'] = gdf_mel['Tot_P_P'] / gdf_mel['AREA_ALBER']

In [9]:
#Replace positive and negative infinity values with NaN in the GeoDataFrame
gdf_mel = gdf_mel.replace([np.inf, -np.inf], np.nan)

In [10]:
# Remove rows with NaN values for population and pop density
gdf_mel.dropna(subset=['Pop_density', 'Tot_P_P'], how='any', inplace=True)

In [11]:
# Set the location (Melbourne city) to be fetched from OSM
location_mel = "City of Melbourne, Australia"

# Fetch the boundary of Melbourne city from OSM using geocode
melbourne_boundary = ox.geocode_to_gdf(location_mel)

#covert the crs to 3112
melbourne_boundary = melbourne_boundary.to_crs(epsg=3112)

# Clip your GeoDataFrame to the Melbourne city boundary
gdf_mel = gpd.overlay(gdf_mel, melbourne_boundary, how='intersection')


In [13]:
# save as a geopackage for use in final map
gdf_mel.to_file('/content/drive/MyDrive/FinalProject/Data/Pop_Density_Data/Mel/processed/pop_density.gpkg', driver='GPKG')

In [12]:
# Create a Folium map centered at a specific location
map_center = [-37.815, 144.94] # Specify the center coordinates (e.g., [latitude, longitude])
m = folium.Map(location=map_center, zoom_start=12)  # You can adjust the zoom level as needed

# Add the GeoDataFrame to the map and specify the column to use for coloring
folium.Choropleth(
    geo_data=gdf_mel,
    data=gdf_mel,
    columns=['SA1_CODE_2', 'Pop_density'],
    key_on="feature.properties.SA1_CODE_2",  # The key for the column in your GeoDataFrame
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Pop density (people per km^2)',   #Label the legend
    name='Melbourne Pop Density'  # Rename the LayerControl title here
).add_to(m)

# add the layer control to turn on/off
folium.LayerControl().add_to(m)

m

In [ ]:
# Creating make of pop density to show in report

# Define the column name to use for coloring the map
attribute_column = 'Pop_density'  # Replace with your actual attribute column name

gdf_mel.plot(column=attribute_column, legend=True)
plt.title('Melbourne SA1 Population Density (people per km^2)')
plt.savefig('Mel population_density_map.png', dpi=300, bbox_inches='tight')

plt.show()